In [1]:
import pandas as pd

train_agg_filename = r'../train/train_agg.csv'
train_flg_filename = r'../train/train_flg.csv'
train_log_filename = r'../train/train_log.csv'
test_agg_filename = r'../test/test_agg.csv'
test_log_filename = r'../test/test_log.csv'


In [2]:
train_agg = pd.read_csv(train_agg_filename, sep='\t')
train_flg = pd.read_csv(train_flg_filename, sep='\t')
train_log = pd.read_csv(train_log_filename, sep='\t')
test_agg = pd.read_csv(test_agg_filename, sep='\t')
test_log = pd.read_csv(test_log_filename, sep='\t')


In [3]:
df_log = pd.concat([train_log, test_log]).reset_index(drop=True)


In [4]:
df_log['OCC_TIM'] = pd.to_datetime(df_log['OCC_TIM'])
df_log['weekday'] = df_log['OCC_TIM'].dt.weekday  #数目

In [5]:
df_log_sum=df_log.groupby(['USRID']).size().reset_index()
df_log_sum.columns=['USRID','LOG_SUM']

In [6]:
df_log_WD=df_log.groupby(['USRID','weekday']).size().unstack().add_prefix('WD_').fillna(0).reset_index()

In [7]:
df_log_unstack = df_log.groupby(['USRID', 'weekday', 'TCH_TYP']).size().unstack().add_prefix(
    'TY_').unstack().add_prefix('WD_').fillna(0).reset_index()


In [8]:
                                        df_log_unstack.head()

TCH_TYP USRID WD_TY_0                                    WD_TY_2            \
weekday          WD_0  WD_1  WD_2  WD_3  WD_4  WD_5 WD_6    WD_0 WD_1 WD_2   
0           2     0.0   8.0   1.0   0.0   0.0   0.0  0.0     0.0  0.0  0.0   
1           3    32.0  33.0  30.0  11.0  27.0  24.0  0.0     0.0  0.0  0.0   
2           4     0.0  18.0   0.0   0.0   0.0   0.0  0.0     0.0  0.0  0.0   
3           5     2.0  18.0   0.0   0.0   0.0   0.0  0.0     0.0  0.0  0.0   
4           7     0.0   8.0  44.0   0.0  16.0   8.0  0.0     0.0  0.0  0.0   

TCH_TYP                      
weekday WD_3 WD_4 WD_5 WD_6  
0        0.0  0.0  0.0  0.0  
1        0.0  0.0  0.0  0.0  
2        0.0  0.0  0.0  0.0  
3        0.0  0.0  0.0  0.0  
4        0.0  0.0  0.0  0.0

In [11]:
df_log['EVT_LBL_1'] = df_log['EVT_LBL'].apply(lambda x: x.split('-')[0])
# df_log['EVT_LBL_2'] = df_log['EVT_LBL'].apply(lambda x: x.split('-')[1])
# df_log['EVT_LBL_3'] = df_log['EVT_LBL'].apply(lambda x: x.split('-')[2])

# df_log['EVT_LBL_2'] = df_log['EVT_LBL'].apply(lambda x: x.split('-')[0]+'-'+x.split('-')[1])
# df_log['EVT_LBL_3'] = df_log['EVT_LBL'].apply(lambda x: x.split('-')[0]+'-'+x.split('-')[1]+'-'+x.split('-')[2])


In [12]:
df_log.head()

,USRID,EVT_LBL,OCC_TIM,TCH_TYP,weekday,EVT_LBL_1
0,10002,163-577-913,2018-03-22 16:31:44,0,3,163
1,10002,163-578-914,2018-03-22 16:31:18,0,3,163
2,10002,259-924-1525,2018-03-22 16:31:15,0,3,259
3,10002,326-1040-1677,2018-03-06 12:08:51,0,1,326
4,10002,326-1041-1678,2018-03-09 14:40:22,0,4,326


In [13]:
df_log=df_log.sort_values(by=['USRID','OCC_TIM']).reset_index(drop=True) #时间排序

In [15]:
df_log['LAST_EVT_LBL_1']=df_log.groupby(['USRID'])['EVT_LBL_1'].shift(1)


In [16]:
df_log['LBL_1']=df_log['LAST_EVT_LBL_1']+'-'+df_log['EVT_LBL_1']


In [17]:
df_log_LBL_1 = df_log.groupby(['USRID', 'EVT_LBL_1']).size().unstack().add_prefix('EVT_LBL_1_').reset_index().fillna(0)



In [19]:
df_log_LBL_1.head()

EVT_LBL_1,USRID,EVT_LBL_1_0,EVT_LBL_1_10,EVT_LBL_1_102,EVT_LBL_1_139,EVT_LBL_1_162,EVT_LBL_1_163,EVT_LBL_1_181,EVT_LBL_1_257,EVT_LBL_1_259,...,EVT_LBL_1_372,EVT_LBL_1_38,EVT_LBL_1_396,EVT_LBL_1_438,EVT_LBL_1_460,EVT_LBL_1_508,EVT_LBL_1_518,EVT_LBL_1_520,EVT_LBL_1_540,EVT_LBL_1_604
0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3,12.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,3.0,...,2.0,8.0,4.0,0.0,0.0,0.0,0.0,33.0,6.0,7.0
2,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,7,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,6.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0


In [20]:
df_log_LBL_1.to_csv('../final_feature/lbl_one_shift.csv',index=None,sep='\t')